# Receiver comparison with an over the air connection


## works, but still a first draft

In [ ]:
from config import *
from OFDM_SDR_Functions_torch import *
import torch
from models_local import *
from OFDM_SDR_Functions_torch import *
from SDR_Pluto import *
import random

use_sdr = True

device = torch.device("cuda:0" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))

# Display the selected device
print(device)

# Test tensor creation on the selected device
if device.type != "cpu":
    x = torch.ones(1, device=device)
    print(x)

device = "cpu" # Force CPU for now, trouble with converting complex tensors to mps

In [ ]:
# for SDR
SDR_TX_Frequency = int(436e6) # SDR TX frequency
tx_gain_min = -50 # sdr tx max gain
tx_gain_max = 0 # sdr tx min gain
rx_gain = 10 # sdr rx gain


# in case SDR not available, for channel simulation
ch_SINR_min = 0 # channel emulation min SINR
ch_SINR_max = 50 # channel emulation max SINR

In [ ]:
mapping_table_QPSK, de_mapping_table_QPSK = mapping_table(2) # mapping table QPSK (e.g. for pilot symbols)
mapping_table_Qm, de_mapping_table_Qm = mapping_table(Qm, plot=False) # mapping table for Qm

TTI_mask_RE = TTI_mask(S=S,F=F, Fp=Fp, Sp=Sp, FFT_offset=FFT_offset, plotTTI=False)
pilot_symbols = pilot_set(TTI_mask_RE, Pilot_Power)


Load the model and wights

In [ ]:

model = RXModel(Qm).to(device)

# Load the model architecture and weights
checkpoint_path = 'data/rx_model_10.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
# start the SDR
if use_sdr: 
    SDR_1 = SDR(SDR_TX_IP="ip:192.168.1.10", SDR_TX_FREQ=SDR_TX_Frequency, SDR_TX_GAIN=-80, SDR_RX_GAIN = 0, SDR_TX_SAMPLERATE=SampleRate, SDR_TX_BANDWIDTH=F*SCS*2)
    SDR_1.SDR_TX_start()

In [ ]:
def create_OFDM_data():
    pdsch_bits, pdsch_symbols = create_PDSCH_data(TTI_mask_RE, Qm, mapping_table_Qm, power=PDSCH_power)
    Modulated_TTI = RE_mapping(TTI_mask_RE, pilot_symbols, pdsch_symbols, plotTTI=False)
    TD_TTI_IQ = FFT(Modulated_TTI)
    TX_Samples = CP_addition(TD_TTI_IQ, S, FFT_size, CP)
    zeros = torch.zeros(leading_zeros, dtype=TX_Samples.dtype)
    TX_Samples = torch.cat((zeros, TX_Samples), dim=0)
    return pdsch_bits, TX_Samples

pdsch_bits, TX_Samples = create_OFDM_data()

In [ ]:
def radio_channel(use_sdr, tx_signal, tx_gain, rx_gain, ch_SINR):
    if use_sdr:
        SDR_1.SDR_gain_set(tx_gain, rx_gain)
        SDR_1.SDR_TX_send(SAMPLES=tx_signal, max_scale=1, cyclic=True)
        rx_signal = SDR_1.SDR_RX_receive(len(tx_signal)*4)
        SDR_1.SDR_TX_stop()
    else:
        rx_signal  = generate_cdl_c_impulse_response(tx_signal = tx_signal, num_samples=16, sampling_rate=SampleRate, SINR=ch_SINR, repeats = 4, random_start=True)
    return rx_signal

In [ ]:
# remove DC and FFT offsets from TTI mask_RE and add third dimension size of Qm, and expand TTI mask values into the third dimension
TTI_mask_RE_3d = TTI_mask_RE[:, FFT_offset:-FFT_offset]
middle_index = TTI_mask_RE_3d.size(1) // 2
TTI_mask_RE_3d = torch.cat((TTI_mask_RE_3d[:, :middle_index], TTI_mask_RE_3d[:, middle_index + 1:]), dim=1)
TTI_mask_RE_3d = TTI_mask_RE_3d.unsqueeze(-1).expand(S, F-1, Qm)

number_of_testcases = 1000

SINR2BER_table = np.zeros((number_of_testcases,5))
wrongs = []
wrongs_ls = []

for i in range(number_of_testcases):
    ch_SINR = int(random.uniform(ch_SINR_min, ch_SINR_max))
    tx_gain_i = int(random.uniform(tx_gain_min, tx_gain_max))
    RX_Samples = radio_channel(use_sdr=use_sdr, tx_signal = TX_Samples, tx_gain = tx_gain_i, rx_gain = rx_gain, ch_SINR = ch_SINR)
    symbol_index=sync_TTI(TX_Samples, RX_Samples, leading_zeros)
    SINR_sig = SINR(RX_Samples, 100, symbol_index)
    RX_NO_CP = CP_removal(RX_Samples, symbol_index, S, FFT_size, CP, plotsig=False)
    RX_NO_CP = RX_NO_CP / torch.max(torch.abs(RX_NO_CP))
    OFDM_demod = DFT(RX_NO_CP, plotDFT=False)
    OFDM_demod = OFDM_demod / torch.max(torch.abs(OFDM_demod)) # normalize DFT'd signal for NN input


    ## NN-receiver
    # create pilot matrix as in the TTI mask
    z, pilot_symbols_map = get_pilot_symbols_raw(TTI_mask_RE, OFDM_demod) # get pilot symbols
    pilot_symbols_map = remove_fft_Offests(pilot_symbols_map, F, FFT_offset) # remove FFT offsets
    pilot_symbols_map = torch.cat((pilot_symbols_map[:, :F//2], pilot_symbols_map[:, F//2 + 1:]), dim=1).unsqueeze(0)# remove DC
    #plt.imshow(torch.abs(pilot_symbols_map.squeeze()))
    
    # create PDSCH matrix, the same shape
    z2, pdsch_symbols_map = get_payload_symbols_raw(TTI_mask_RE, OFDM_demod) # get PDSCH symbols
    pdsch_symbols_map = remove_fft_Offests(pdsch_symbols_map, F, FFT_offset) # remove FFT offsets
    pdsch_symbols_map = torch.cat((pdsch_symbols_map[:, :F//2], pdsch_symbols_map[:, F//2 + 1:]), dim=1).unsqueeze(0) # remove DC
    #plt.imshow(torch.abs(pdsch_symbols_map.squeeze()))

    OFDM_NN = model((pdsch_symbols_map.to(device), pilot_symbols_map.to(device)))
    binary_predictions = torch.round(OFDM_NN)
    binary_predictions = torch.tensor(binary_predictions.squeeze()[TTI_mask_RE_3d==1].flatten().cpu(), dtype=torch.int8)

    # Calculate Bit Error Rate (BER) for the NN-receiver
    error_count = torch.sum(binary_predictions.flatten() != pdsch_bits.flatten()).float()  # Count of unequal bits
    new_wrongs = (binary_predictions.flatten() != pdsch_bits.flatten()).float().tolist()
    wrongs.append(new_wrongs)


    error_rate = error_count / len(pdsch_bits.flatten())  # Error rate calculation
    BER_NN = torch.round(error_rate * 1000) / 1000  # Round to 3 decimal places
    

    # LS/ZF receiver
    H_estim = channelEstimate_LS(TTI_mask_RE, pilot_symbols, F, FFT_offset, Sp, OFDM_demod, plotEst=False)
    OFDM_demod_no_offsets = remove_fft_Offests(OFDM_demod, F, FFT_offset)
    equalized_H_estim = equalize_ZF(OFDM_demod_no_offsets, H_estim, F, S)
    QAM_est = get_payload_symbols(TTI_mask_RE, equalized_H_estim, FFT_offset, F, plotQAM=False)
    PS_est, hardDecision = Demapping(QAM_est, de_mapping_table_Qm)
    bits_est = PS(PS_est)
    error_count = torch.sum(bits_est != pdsch_bits.flatten()).float()  # Count of unequal bits
    error_rate = error_count / bits_est.numel()  # Error rate calculation
    BER_LSZF = torch.round(error_rate * 1000) / 1000  # Round to 3 decimal places
    new_wrongs_ls = (bits_est != pdsch_bits.flatten()).float().tolist()
    wrongs_ls.append(new_wrongs_ls)


    # Save results
    SINR2BER_table[i,0] = round(tx_gain_i,1)
    SINR2BER_table[i,1] = round(rx_gain,1)
    SINR2BER_table[i,2] = round(SINR_sig[0],1)
    SINR2BER_table[i,3] = BER_LSZF
    SINR2BER_table[i,4] = BER_NN
    

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(wrongs)

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.plot(np.sum(wrongs, axis=0))

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.plot(np.sum(wrongs_ls, axis=0))

In [ ]:
import pandas as pd

df = pd.DataFrame(SINR2BER_table, columns=['txg', 'rxg', 'SINR', 'BER_LSZF', 'BER_NN'])
save_plots = False
df['SINR_binned'] = pd.cut(df['SINR'], bins=range(-10, 40, 2), labels=range(-9, 39, 2))
df_grouped = df.groupby('SINR_binned', observed=False).mean()

plt.figure(figsize=(8, 4))
plt.plot(df['SINR'], df['BER_LSZF'], 'o', alpha=0.15, markersize=3, c='blue')
plt.plot(df['SINR'], df['BER_NN'], 'o', alpha=0.15, markersize=3, c='red')

plt.xlabel('SINR (dB)')
plt.ylabel('Uncoded BER')
plt.plot(df_grouped['SINR'], df_grouped['BER_LSZF'], '-o', color='blue', label='LS/ZF')
plt.plot(df_grouped['SINR'], df_grouped['BER_NN'], '-o', color='red', label = 'NN')
plt.legend()
plt.title(f'OFDM System Uncoded Performance\n Qm={Qm}, FFT_size={FFT_size}, F={F}, S={S}, Fp={Fp}, Sp={Sp}, CP={CP}, SCS={SCS}\nPilot_P={Pilot_Power}, PDSCH_P={PDSCH_power}, freq={SDR_TX_Frequency}Hz')
plt.xlim(0, 30)
plt.yscale('log')
plt.ylim(1e-3, 1)   
plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
if save_plots:
    plt.savefig('pics/Performance_LS_nn.png')
plt.show()